# Class 3 - 18.3.18

## Environments

Environments are the Pythonian solution to the "path" problem of MATLAB and other languages.

The basic idea is that every Python project you have has an isolated environment, in which it resides. This environment contains a Python interpreter and libraries that are needed for that specific project, but aren't necessarily required by other projects. 

While it might not seem _that_ important at first, when your project has multiple dependencies on other libraries, asserting compatibility between all components become tricky. This point will be much clearer as the course progresses, so in the mean time just stick with me.

Your different Python environments are managed by a package manager, and the best one (as of early 2018) is `conda`, by Anaconda Inc. (previously Continuum Analytics). conda has two versions, and we'll be working with the slimmer Miniconda distribution (for Python 3.6), as explained in `PythonSetup.md`.

### How to create an environment?

After you've installed Miniconda and added it to your path, write in the command-line `conda create --name my_env python=3.6`. This creates a new Python interpreter named `my_env`. To make sure we're working with this specific interpreter, we have to activate the environment:
`activate my_env`. In Mac\Linux, we write `source activate my_env`.

From now on, until we write `deactivate` (or `source deactivate`), all Python-related operations will be performed with the `my_env` interpreter in mind. If we have other environments they will not be affected by any change made as long as `my_env` is active. We can also define environment variables which will obviously be specific to this environment alone.

The environment manager we use is called Miniconda, and while Python comes with an environment manager of its own (`virtualenv`), `conda` is the more popular option, especially for scientific Python. `conda` manages dependencies in a more strict fashion than `pip`, and can be used to manage the dependencies of other applications. Miniconda is a "bare-bones" installation, so it doesn't bloat your computer with unneeded packages, allowing for manual control of each package and environment. It also serves for a better learning experience than Anaconda.

For our class you should probably create one environment for the entire semester. While I can't really verify that you're indeed working with environments, I promise you that it's a very useful habit that will save you a great amount of trouble later on, probably without you even noticing it.

![Summary of environments and namespaces](extra_material/envs.png)

## Exercise

- Create a new environment in your computer.
- Create a new folder that will contain a mock Python project, and make this folder a version-controlled one.
- Add a couple of `.py` files and add to them to the list of tracked files. Make sure to use the right folder structure, as outlined above.
- In one of these `.py` files we'll try out list comprehensions:
    - A Cartesian product is the list of all combinations of two, or more, sequences. For example, the product of `AB` and `abc` is `Aa, Ab, Ac, Ba, Bb, Bc`. 
    - Using a nested for loop, print out a list of tuples of the Cartesian product of `ABC` and `abcd`. 
    - Using list comprehension print out the exact same output.
- Run the function that does the comprehension from a different file using `import`.
- Install the package `pytz` using `pip` and make sure you're able to `import` it.
- Assuming this project will contain raw `.tif` data, add a `.gitignore` file that disregards the raw data.
- Add an MIT license to the repo, and a basic `README.md`.
- Publish this project to your GitHub account.
- If you already have VS Code or PyCharm on your computer, make sure you know how to link these programs to this specific environment.

### Exercise solution below...

## Scripts and Functions 
#### (Code examples in `import_demonstration` folder)

If you're familiar with MATLAB there's a good chance that you've written a script before. A script is a file which is run sequentially, while using other functions and definitions. Python supports scripts as well, as can be seen in `main.py`.

However, in the Python world people usually prefer to stay away from scripts. This is due to a number of reasons, the most important one being that running a `main()` function as it is easy as running a `main.py` script. You can see examples of a procedural and script-like approach in the `main.py` file, but keep in mind the the script version is discouraged.

If you wish to run a file full with functions from the command line, or from your IDE, you should include the following lines:
```python
if __name__ == '__main__':
    run_main()
```

Every Python file which is being run has a caller. If this file was run directly from the Python interpreter its `__name__` will be `'__main__'`. This `if` statement basically tells the Python interpreter "Start from here", and is the conventional way to run Python procedures.

In this course you're highly encouraged to divide your code into many small functions and methods in well-defined compact classes. Each method should have a single purpose, documented in its docstring. Each class should have a logical structure that envelopes its methods and attributes in a sensible way.

Beware of God classes, or God scripts and functions. These are monolithic objects that encompass the entirety of your application, and are very hard to reason about.

Another important reason to partition our code into many small bits is _unit testing_, which we'll cover later on in the course.

# Object-Oriented Programming

## Introduction

There are three main programming paradigms in use in mainstream programming languages:
* Functional
* Procedural
* Object-oriented

While a _functional_ paradigm is very interesting, we'll not be discussing it in this course. You can read about Haskell, OCaml, F# and other functional programming languages wherever you get your information from.

The _procedural_ paradigm is the most widely used paradigm... in the academia. And it's probably the one you're most familiar with from your work in Matlab. 

Confusingly, this paradigm is based around _functions_ (procedures):

In [1]:
# We could write:
l1 = [1, 2, 3]
l2 = [4, 5, 6]
result = []
for item1, item2 in zip(l1, l2):
    result.append(item1 * item2)
result

[4, 10, 18]

In [2]:
# And when we have more lists to multiply we'll again write:
l3 = [10, 20, 30]
l4 = [40, 50, 60]
result2 = []
for item3, item4 in zip(l3, l4):
    result2.append(item3 * item4)
result2

[400, 1000, 1800]

But we're seeing a pattern here. The important DRY principle requires "Don't Repeat Yourself", so we define a function to replace these two implementations:

In [6]:
def list_multiplier(l1, l2):
    """
    Multiply two lists element-wise.
    Returns a list with the result
    """
    result = []
    for item1, item2 in zip(l1, l2):
        result.append(item1 * item2)
    
    return result

This new _procedure_ does one thing, and one thing only. This is what's so powerful about it.

Procedural programming allows us to group and order our code base into small units, called functions or _procedures_, that have a specific, defined task.

It usually contains a "wrapper" script that defines the order of running for these functions:
```python
# my_wrapper_script.py
def run_pipeline(foldername):
    """ Main data pipeline script """
    data = get_user_input(foldername)
    data_without_fieldnames = extract_fieldnames(data)
    columnar_data = generate_columns(data_without_fieldnames, num_of_columns)
    # ...
    # At the end of the file it will contain:
if __name__ == '__main__':
    foldername = r'/path/to/folder'  # raw string
    result = run_pipeline(foldername)
    print(result)
```

You should be extremely decisive and eliminate repeating code. It's perhaps the most common source for errors in scientific computing, and it may bite you any of these ways:

- Encapsulation:
```python
# String concatenation
first_string = 'abcd'
second_string = 'efgh'
concat = first_string + second_string[:-1] + 'zzz'  # you suddenly remember that you wish to exclude 
# the last character in "second_string" and add the 'zzz' sequence at the end.
# Program continues...
# ...
third_string = 'poiu'
fourth_string = 'qwer'
concat2 = third_string + fourth_string + 'zzz' # you wish to achieve the same goal in this 
# concatenation - but you forgot that you excluded the last character of the second string.
```
    The moment you realized that you have a recurring operation on strings - you have to encapsulate it in a function - be _ruthless!_

- Parametrization

```python
def process_data(data):
    scaled_data = data * 0.3  #  what is 0.3 exactly? Parametrize it.
    
def process_data(data, param=0.3):
    scaled_data = data * param

```

But this is usually not enough. When calling the `process_data` parameterize the `param` variable as well:

```python
data = b * c - 1 + a
process_data(data, 0.4)
# Script continues...
process_data(data2, 0.5)  # Perhaps you really did wish to call "process_data" with two different
# parameters, but it's more likely that you decided that 0.5 was too high, so you changed it to 0.4
# in the first call, but forgot that you had a second call. This parameter should appear somewhere at
# the top of your script.
```

## Alternatives

While procedural programming works great most of the time, it can sometime be inferior to other paradigms, namely _object-oriented programming._

But what do I mean by _inferior_? Obviously, all programs can be written successfully without leaving the safe confines of procedural programming.

While this statement holds true, sometimes our _mental model_ of the task at hand fits an object-oriented paradigm more naturally.

### Classes and Objects

Classes are used-defined types. Just like `str`, `dict`, `tuple` and the rest of the standard types, Python allows us to create our own types.

Objects are _instances_ of classes, they're an instance of a type we made. Actually, _all_ instances of _all_ types are objects in Python. It means that every variable and function in Python are, by themselves, an instance of a type. A function you make is an instance of the `function` type, for example. We'll get to this during later stages of the course.

Classes are a type of abstraction we create with our code. A variable is the most simple type of abstraction - it's a _thing_ that is closely tied to a "real value" in a very simple relationship: My variable $x$ represnts the value $y$. 

Classes are more abstract - they don't relate to a specific value directly, but rather they try to convey an idea of an object.

### A Classic - The Point

To show what we mean by "our own type", we'll define the `Point` type.

What is a point? What is composed of?
* In a 2D space it's a pair of values, $(x, y)$, specifying a location on a grid.
* $x$ and $y$ are the coordinates of the point.
* Points have special relations to other points and to the space they reside in.

From these three simple observations, we expect our `Point` type to include both data about its coordinates, and functions, or _methods_, used to interact with the grid and\or other points. 

An object usually bundles together data and methods to use the data.

We wish to express these abstract ideas in our code. It might seem like a lot of code at the start ("boilerplate" code) - but it will be worth it - certainly for more complex classes.

In [5]:
# Introducing the class keyword:
class Point:
    """Represents a point in a 2D space"""

# Notice the capital letter

In [6]:
Point
# A new type is born in __main__

__main__.Point

In [7]:
# The keyword Point is now a factory to create new Points.
# To make one, we have to call it like we do with a function:
blank = Point()
blank
# We call this instantiation, as blank is an instance of Point

In [8]:
# Assign the point's data in the form of coordinates
blank.x = 1.0
blank.y = 0.0

In [8]:
# x and y are now attributes of our class:
blank.x

1.0

The notation `.x` means "go the instance `blank` of the class `Point` and find the value `x` refers to."

There's no conflict between a variable named `x` and the attribute `x`.

The attribute can be used anywhere:

In [9]:
# Simple statements
print(1 + blank.x)

2.0


In [10]:
# Printouts
"A case of a pointy Point at {}".format((blank.x, blank.y))

'A case of a pointy Point at (1.0, 0.0)'

In [11]:
# Arguments to functions
def print_point(p):
    """Print a Point object"""
    print("{}".format((p.x, p.y)))


In [11]:
print_point(blank)

(1.0, 0.0)


### Exercise:
Write a function `distance_between_points(p1, p2)` that takes two points and returns the Cartesian distance between them.

### Exercise solution below...

In [10]:
import math


def distance_between_points(p1, p2):
    """ Calculate Cartesian distance between points """
    return math.sqrt((p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2)

blank2 = Point()
blank2.x = 1.
blank2.y = 1.
dist = distance_between_points(blank, blank2)
dist

1.0

### Rectangles

If we wish to model a rectangle, the first thing that should now be clear is that a rectangle is best modelled with a class (at least in Python). 

Deciding on the exact content of our type is sometimes not straight-forward. How would you implement a rectangle?

Modelling a rectangle can be done in the following ways:

* We can decide to define it with a point and two sides.
* We can choose the point to be a corner or its center.
* We can also use two opposing points.

Here's we'll go with the first option, with the Point being the corner.

In [12]:
class Rectangle:
    """
    Rectangle model
    Attributes:
    corner - bottom left corner, of type Point
    height - float
    width - float
    """

In [14]:
rect = Rectangle()
rect.width = 100.0
rect.height = 200.0
rect.corner = Point()
rect.corner.x = 0.0
rect.corner.y = 0.0

rect

In [18]:
# We can return instances of classes (just like we do with instances of dictionaries...)
def find_center(box):
    """ 
    Return a Point to the center of the Rectangle box
    """
    p = Point()
    p.x = box.corner.x + box.width / 2
    p.y = box.corner.y + box.height / 2
    return p

In [19]:
center = find_center(rect)
print_point(center)

(50.0, 100.0)


In [ ]:
# Objects are mutable - we can change their attributes:
    
print(rect.corner.x)
rect.corner.x += 100
print(rect.corner.x)

In [21]:
def grow_rectangle(rect, dwidth, dheight):
    """ Take a Rectangle instance and grow it by (dwidth, dheight) """
    rect.width += dwidth
    rect.height += dheight
    # No need to return the instance

In [23]:
rect.width, rect.height

(100.0, 200.0)

In [24]:
grow_rectangle(rect, 100, 100)
rect.width, rect.height

(200.0, 300.0)

### Class Methods

We really haven't done object-oriented programming yet. Our objects currently contain only *data*, in the form of their attributes.

To go one step closer we must learn of _methods_.

Methods are functions bound to objects, describing actions they can do, or that can be done to them. For example, a real-world car can drive. So a `Car` object should have a `drive()` method attached to it. It should also have a `park()` method, and a couple of attributes, like `number_of_wheels`, `manufacturer` and `model`.

As we'll see in a second, the only differentce between methods and functions is that the former are always attached to an object, and they only make sense when attached to that specific object. A `park()` method has no meaning when we try to run it on a `Rectangle()`.

In [25]:
# Basic example
class LightBulb:
    """
    A light bulb to shed light on us all.
    Attributes: is_on, power
    Methods: turn_on, turn_off
    """
    # Here comes the methods definitions:
    def turn_on(self):
        """ Turns a light bulb on """
        self.is_on = True
        print("Shining strong!")
    
    def turn_off(self):
        """ Turns a light bulb off """
        self.is_on = False
        print("Good night!")
    

In [28]:
bulb = LightBulb()
bulb.is_on = False
bulb.turn_on()

Shining with 40 power!


In [29]:
bulb.turn_off()

Good night!


In [30]:
# As we see, the only thing that changed was the indentation, and the method is now "bound"
bulb.turn_on

<bound method LightBulb.turn_on of <__main__.LightBulb object at 0x00000240D54CC668>>

The active agents here are the _objects_, not the functions. Instead of `turn_on(bulb)` we have the bulb "turning itself on": `builb.turn_on()`.

In general, most functions that take an instance as one of their parameters should be a candidate for becoming a method, bound to that object, since you might need it later on for other instances as well.

In [32]:
# Methods can be called inside other methods:
# I'll redefine LightBulb for the sake of completeness:
# Basic example
class LightBulb:
    """
    A light bulb to shine on us.
    Attributes: is_on, power
    Methods: turn_on, turn_off, remove
    """
class LightBulb:
    """
    A light bulb to shed light on us all.
    Attributes: is_on, power
    Methods: turn_on, turn_off
    """
    # Here comes the methods definitions:
    def turn_on(self):
        """ Turns a light bulb on """
        self.is_on = True
        print("Shining strong!")
    
    def turn_off(self):
        """ Turns a light bulb off """
        self.is_on = False
        print("Good night!")
        
    # -----The new method:-----
    def remove(self):
        """ Disconnect the bulb from its socket """
        self.turn_off()
        print("Twisting the bulb counter-clockwise")


This `self` call is the self-reference of the object to itself. All methods and attributes must be defined with the `self` parameter as their first parameter.

### The `__init__` method

Classes have several special methods attached to them. While some are out the course's scope, a special method that __does__ deserve a closer look is the `__init__()` method. It has two underscores before and after its name, making it a "dunder" method, short for "double underscore".

The `__init__()` methods allows us to define our class' attributes inside the class definition:

In [1]:
class NewClass:
    def __init__(self, attr1, attr2=20, attr3='abc'):
        self.attr1 = attr1
        self.attr2 = attr2
        self.attr3 = attr3
# attr1 has no default, forcing the user to define it by him\herself.

Most chances are that the first method you'll write for a newly defined class is the `__init__()` method. 

Another interesting _dunder_ method is the `__str__()` method, which defines what the class will print when invoked using the `print(class_instance)` command. For example:

In [2]:
class ShoppingList:
    def __init__(self, vegetables=10, fruits=5, bread=1):
        self.vegetables = vegetables
        self.fruits = fruits
        self.bread = bread
    
    def __str__(self):
        str_to_print = """Shopping List:
        Vegetabels: {}
        Fruits: {}
        Bread: {}
        Total items: {}
        """.format(self.vegetables, self.fruits, self.bread,
                   self.vegetables + self.fruits + self.bread)
        return str_to_print

In [3]:
shop_list = ShoppingList(vegetables=5, fruits=1, bread=3)

In [5]:
print(shop_list.fruits)

1


In [6]:
print(shop_list)

Shopping List:
        Vegetabels: 5
        Fruits: 1
        Bread: 3
        Total items: 9
        


The `__str__()` method is very useful for debugging purposes.

### Operator Overloading

One of the most interesting properties of Python is operator overloading (although it's not unique to Python). It means that we can force our self-declared types (i.e. classes) to behave in a certain way with the standard mathematical operations.

We'll use the ShoppingList class as an example. Say we want to __add__ two different shopping lists. Naively, we might just try the following:

In [8]:
shoplist1 = ShoppingList()
shoplist2 = ShoppingList()
print(shoplist1 + shoplist2)

TypeError: unsupported operand type(s) for +: 'ShoppingList' and 'ShoppingList'

To us, this expression seems completely fine - adding two shopping lists should just concatenate the items one after the other. The fact that it's a very readable line of code makes it a _good_ line of code, since you have to remember that we write code for humans to read, not computers.

Unfortunately, Python can't add two shopping lists because it was never taught how to do that. Luckily, we can override the behavior of the addition operator, by defining the `__add__()` method in the class definition:

In [14]:
class ShoppingList:
    def __init__(self, vegetables=10, fruits=5, bread=1):
        self.vegetables = vegetables
        self.fruits = fruits
        self.bread = bread
    
    def __str__(self):
        str_to_print = """Shopping List:
        Vegetabels: {}
        Fruits: {}
        Bread: {}
        Total items: {}
        """.format(self.vegetables, self.fruits, self.bread,
                   self.vegetables + self.fruits + self.bread)
        return str_to_print
    
    # ----- New method below: ------
    def __add__(self, other):
        """ Concatenate two shopping lists """
        try:
            new_list = ShoppingList(
                vegetables=self.vegetables + other.vegetables,
                fruits=self.fruits + other.fruits,
                bread=self.bread + other.bread
            )
        except AttributeError as e:
            print("AttributeError, Addition is supported only between two ShoppingLists")
            
        return new_list

In [17]:
# Now we can safely add two ShoppingList() instances:
shoplist1 = ShoppingList()
shoplist2 = ShoppingList()
added_shoplist = shoplist1 + shoplist2
print(added_shoplist)

Shopping List:
        Vegetabels: 20
        Fruits: 10
        Bread: 2
        Total items: 32
        


In [22]:
print(shoplist1)b

Shopping List:
        Vegetabels: 10
        Fruits: 5
        Bread: 1
        Total items: 16
        


In [18]:
# Addition of something other than a ShoppingList will result in an AttributeError:
shoplist1 + 1
# It is more Pythonic to try and catch the exception than to check in advance
# the types of all variables involved.

AttributeError: Addition is supported only between two ShoppingLists

We can overload all operators to make our classes behave as one would intuitively expect them to.

We usually don't write new dunder methods. That is, Python pre-defines the relevant dunder methods for us, and we just re-use their names. Generally you shouldn't implement some random method you thought of as a dunder method.

### Summary

OOP is the most important programming paradigm nowadays, and you should be very familiar with it. Some problems fit this paradigm hand in glove, and the best example is GUI programming. However, it's not the "ultimate" answer to any design difficulty you have. Some problems _can_ be solved by using intricate objects and multiple inheritance, but in reality they're much simpler when solved using procedural design. Remember to write code that humans, and especially your future self, can read and understand.

With that being said, throughout the semester I prefer you write _too many_ objects over writing _too few,_ as it indicates you feel comfortable in the object-oriented world.

## Two Exercises

### The Fraction Class

Create a `Fraction()` class with only the basic attributes. Assume that the inputs are numbers.

a. Override some operators so that you can add a Fraction to an integer (`Fraction() + int` only, not `int + Fraction()`).

b. Override the operator that allows you to check which fraction is larger between the two.

### The Path Class

Create a `Path()` class, symboling a directory in a mock filesystem. The class should at least one attribute and three methods, not including the mandatory `__init__` method:
- `Path.get_parent()` - returns the name of the parent folder of our Path (not the entire path).
- `Path.get_size()` - returns a number corresponding to the size in KB of the folder, which should depend on the number of files in the folder.
- `Path.set_path(Path())` - Change the current directory to the one given (as a Path() instance).
`

After you've created it, overload the division operator `/` (`__truediv__`) to concatenate two a Path class instance with a string pointing to a folder, i.e.:
```python
>> print(Path("/home") / "/usr")
Path("/home/usr")
```
_Note:_ The filesystem is a _mock_ one, meaning that it shouldn't correspond to the actual file system, but to path-like strings you invent, like the ones above.

You can also assume that the string doesn't contain you own path, i.e. you don't have to deal with cases such as:
`Path("/home") / "/home/usr"`

### Exercises solutions follow...

In [40]:
# Exercise 1
class Fraction:
    """ 
    Models a fraction with a numerator and a denominator, assuming the inputs are numbers
    Attributes:
    num - int, float - numerator
    denom - int, float - denomerator
    value - num/denom
    Methods:
    Can be added and compared to other fractions
    """
    def __init__(self, num, denom):
        self.num = num
        self.denom = denom
        try:
            self.value = self.num/self.denom
        except ZeroDivisonError:
            self.value = None
    
    def __str__(self):
        return f"{self.num}/{self.denom} = {self.value}"
    
    def __add__(self, other):
        """ Left-add a Fraction to an integer """
        return Fraction(self.num + other, self.denom)
    def __gt__(self, other):
        """ Left > between two fractions """
        try:
            return self.value > other.value
        except AttributeError:
            return NotImplemented

In [51]:
f1 = Fraction(1, 2)
f2 = Fraction(3, 4)

print(f1)

print(f1.value)

print("Is f1 bigger?")
print(f1 > f2)

print("Is f1 smaller?")
print(f1 < f2)

print("Are they equal?")
print(f1 == f2)

print("Are they not equal?")
print(f1 != f2)
# Damn, Python's smart.

0.5
1/2 = 0.5
Is f1 bigger?
False
Is f1 smaller?
True
Are they equal?
False
Are they not equal?
True


In [37]:
# Exercise 2 - the Path class
import os


class Path:
    """ 
    File system path to folders, allows the use of "/" to move between folders
    Attributes:
    path - str
    files - list of str
    
    Methods:
    get_parent - returns parent folder as Path
    get_size - returns size of current folder in KB
    set_path(new_path) - changes current path to the .path attribute of new_path
    """
    def __init__(self, path, files=['a.txt', 'b.py', 'c.c']):
        self.path = path
        self.files = files
    
    def get_parent(self):
        """ Returns the name of the parent folder """
        separated_path = self.path.split(os.sep)
        try:
            parent = separated_path[-2]
        except IndexError:
            return None
        else:
            return separated_path[-2]
    
    def get_size(self):
        """ Returns the size in KB of the folder's content """
        return 10 * len(self.files)
    
    def set_path(self, new_path):
        """ Changes current path to new_path """
        try:
            _path, _files = new_path.path, new_path.files
        except AttributeError:
            print("Input variable must be a Path-like instance")
            raise
        else:  # Written like that because we're afraid that the "new_path" variable might have
               # a ".path" attribute, but not a "files" attribute, for example
            self.path = _path
            self.files = _files
    
    def __truediv__(self, other):
        """ 
        Traverse the filesystem using the / sign, assuming that 
        other isn't an absolute path.
        Returns a new instance
        """
        assert type(other) in (str, int)
        new_path = self.path + os.sep + str(other)
        new_files = []
        return Path(path=new_path, files=new_files)

In [38]:
p = Path(r'\home\usr\python\python36', files=['ab.R, l.cpp, py.py'])
print(p.path, p.files)

size = p.get_size()
parent = p.get_parent()
print(f"Size: {size}, parent: {parent}")

\home\usr\python\python36 ['ab.R, l.cpp, py.py']
Size: 10, parent: python


In [22]:
p.set_path(Path(r'\usr\local\bin'))
print(p.path, p.files)

\usr\local\bin ['a.txt', 'b.py', 'c.c']


In [39]:
new_path = p / 'data'
print(new_path.path)

\home\usr\python\python36\data


## Inheritance

One of the most important aspects of object-oriented programming is _inheritance_. Since we're now modeling real-life objects, we can think of the relations between these objects in our code. These relations are sometimes deemed "interfaces", and one of the most important interfaces is inheritance. A simple example should be quite clear:

In [24]:
# The base class
class Person:
    """A base class representing the average Joe (or Jane)"""
    def __init__(self, name='Jane', age=42, gender='F'):
        self.name = name
        self.age = age
        self.gender = gender
    
    def is_old(self):
        """Ageism is wrong"""
        return self.age > 120

In [25]:
woman_a = Person(name='Tammy', age=40, gender='F')
woman_a.is_old()

False

As you can see, we created a `Person()`, which obviously corresponds to a (simplified) real-life person.

Now we'll introduce inheritance, by creating a new class that inherits from `Person()`:

In [26]:
class Student(Person):
    def __init__(self, name='Jane', age=42, gender='F', school='Sagol', gpa=5.):
        super().__init__(name, age, gender)  # initialize the Person, termed super(), for super-class
        self.school = school
        self.gpa = gpa
        
    def try_expel(self):
        """A low GPA will get you kicked"""
        if self.gpa < 4.5:
            self.school = None
            return True

In [16]:
stud = Student('John', 25, 'M', 'Life Sciences', 4.2)

Which methods and attributes does our `stud` have?

In [17]:
# Methods from both the Person and the Student class, of course.
print(stud.name)
print(stud.gpa)
print(stud.school)

John
4.2
Life Sciences


In [18]:
stud.is_old()

False

In [19]:
stud.try_expel()
print(stud.school)

None


Inheritance facilitates code re-use and simpler, clearer mental models of the problem at hand.

A possible issue with inheritance is readability - finding the methods that are associated with the base class can be cumbersome. This is why usually people try to avoid more than a single layer of inheritance.

## Exercise
### Smartphones

Model both a smartphone and a label-specific phone (like iPhone) by using a parent and child class. Have at least 2 methods and 2 attributes for the base class, and at least one unique method for the child class.

One of the methods has to be a `call(phone)` method, designed to call from one phone to the other. When `call()`ing between iPhones, the method should use the FaceTime interface of the two iPhones. Make sure to keep a log of the calls on both phones.

### Exercise solution below...

In [57]:
import time


class Phone:
    """ Base class for all types of mobile phones """
    
    def __init__(self, name, screen_size, num_camera=2):
        self.name = name
        self.screen_size = screen_size
        self.num_of_camera = num_camera
        self.is_on = True
        self.photos = []
        self.calls = {}
    
    def switch_power(self):
        self.is_on = False if self.is_on else True
    
    def take_photo(self):
        """ Take a photo and append it to the photo album """
        self.photo.append([[1, 0], [0, 1]])
    
    def call(self, other):
        """ Call another Phone instance """
        if self.is_on and other.is_on:
            self.calls[other.name] = time.time()
            other.calls[self.name] = time.time()
        else:
            print(f"Phone {other.name} is off.")
            
        return other

class IPhone(Phone):
    """ A more expensive phone, that can call other iPhones using a special call method """
    
    def __init__(self, name, screen_size, num_camera, apple_id):
        super().__init__(name, screen_size, num_camera)
        self.apple_id = apple_id
        self.facetime_calls = {}
    
    def call(self, other):
        """ Overrides the call method from the parent class """
        if self.is_on and other.is_on:
            try:
                self.facetime_calls[other.apple_id] = time.time()
            except AttributeError:
                self.calls[other.name] = time.time()
                other.calls[self.name] = time.time()
            else:
                other.facetime_calls[self.apple_id] = time.time()
        else:
            print(f"Phone {other.name} is off.")
        
        return other
        

In [65]:
regular = Phone(name='lg_v10', screen_size=6)
iphone = IPhone(name='iphone_8', screen_size=5.5, num_camera=3, apple_id='first_iphone_8')
iphone2 = IPhone(name='iphon_X', screen_size=6, num_camera=3, apple_id='second_iphone_X')

# Call from regular phone to iPhone
print(f"Before calling, the log for regular shows: {regular.calls}")
iphone = regular.call(iphone)
print(f"After the call, regular shows {regular.calls} and the iPhone shows {iphone.calls}")

Before calling, the log for regular shows: {}
After the call, regular shows {'iphone_8': 1519839626.4894626} and the iPhone shows {'lg_v10': 1519839626.4894626}


In [66]:
# Two iPhones:
print(f"Before calling, the log for first iPhone shows: {iphone.facetime_calls}")
iphone2 = iphone.call(iphone2)
print(f"After the call, the first iPhone shows {iphone.facetime_calls} and the second shows {iphone2.facetime_calls}")

Before calling, the log for first iPhone shows: {}
After the call, the first iPhone shows {'second_iphone_X': 1519839626.66134} and the second shows {'first_iphone_8': 1519839626.66134}


Object-oriented design requires you to think about the code you're about to write - how to model each object, how to deal with the interfaces between them, how to verify the types of each input, etc.

Because we're trying to model a complex structures, we usually don't succeed in the first try. That's because we become smarter and understand our needs from the model better __only after we've used it.__ Premeditating and debating on the exact way through which two `Phones()` will call each other is important, but we'll usually just __refactor__ our initial model in favor of a better one after a few days of "usage". That's the underlying reason for "alpha" and "beta" versions of software. 

In short, rewriting large parts of an application you designed is expected, since it's a natural and important part of software design - a luxury other engineers rarely have.